In [ ]:
# Librairies
import xarray as xr
import pandas as pd
import numpy as np
import glob
import os

# CONFIGURATION
path_data = "C:/Users/margo/OneDrive/Documents/M1 SSD/Projet_Tutore/Analyse_Meteo/data/raw-yearly-combined/era5_fr_t2m/*.nc"
path_mask = "C:/Users/margo/OneDrive/Documents/M1 SSD/Projet_Tutore/Analyse_Meteo/src/mask_france_boolean.nc"

print("🚀 Démarrage du script de récupération...")

# TRAITEMENT GLOBAL
try:
    # 0. SÉLECTION DES 25 PREMIÈRES ANNÉES
    print("🔍 Sélection des fichiers...")
    fichiers = sorted(glob.glob(path_data))
    print(f"   → {len(fichiers)} fichiers trouvés au total")

    # Prendre les 300 premiers fichiers (25 ans × 12 mois)
    fichiers_25ans = fichiers[:300]
    print(f"   → Sélection de {len(fichiers_25ans)} fichiers (25 premières années)")

    premier_fichier = os.path.basename(fichiers_25ans[0])
    dernier_fichier = os.path.basename(fichiers_25ans[-1])
    print(f"   → Premier : {premier_fichier}")
    print(f"   → Dernier : {dernier_fichier}")

    # Chargement du masque
    print("⏳ Chargement du masque...")
    mask = xr.open_dataset(path_mask)

    # TRAITEMENT PAR ANNÉES (12 fichiers à la fois)
    print("\n📊 Traitement année par année pour éviter la saturation mémoire...")

    moyennes_annuelles = []

    for annee in range(25):
        debut = annee * 12
        fin = debut + 12
        fichiers_annee = fichiers_25ans[debut:fin]

        annee_num = 1950 + annee
        print(f"   → Année {annee_num} ({annee+1}/25)...", end=" ")

        try:
            # Charger une année
            ds_annee = xr.open_mfdataset(
                fichiers_annee,
                engine='netcdf4',
                combine='by_coords'
            )

            # Appliquer le masque
            ds_masked = ds_annee.where(mask['mask'] == 1)

            # Calculer la moyenne spatiale
            ds_mean = ds_masked.mean(dim=['latitude', 'longitude'], skipna=True)

            # Convertir en DataFrame
            df_annee = ds_mean.to_dataframe().reset_index()

            if 'valid_time' in df_annee.columns:
                df_annee = df_annee.rename(columns={'valid_time': 'time'})

            moyennes_annuelles.append(df_annee)

            # Libérer la mémoire
            ds_annee.close()
            del ds_annee, ds_masked, ds_mean

            print("✓")

        except Exception as e_annee:
            print(f"✗ Erreur : {e_annee}")

    # Combiner toutes les années
    print("\n🔗 Fusion des données...")
    df_complet = pd.concat(moyennes_annuelles, ignore_index=True)

    # --- SAUVEGARDE CSV ---
    print("💾 Sauvegarde du CSV...")
    if 't2m' in df_complet.columns:
        df_complet['Temperature_C'] = df_complet['t2m'] - 273.15

        # S'assurer que 'time' existe
        if 'time' not in df_complet.columns:
            df_complet['time'] = df_complet.iloc[:, 0]

        final_df = df_complet[['time', 'Temperature_C']].sort_values('time')
        final_df.to_csv("donnees_meteo_france_25ans.csv", index=False)

        print(f"✅ Fichier CSV créé : 'donnees_meteo_france_25ans.csv'")
        print(f"   → {len(final_df)} mesures sauvegardées")
        print(f"   → Période : {final_df['time'].min()} à {final_df['time'].max()}")
    else:
        print("❌ Erreur : Colonne 't2m' introuvable")
        print(f"   Colonnes disponibles : {df_complet.columns.tolist()}")

    # --- FICHIER 2 : LA CARTE (NetCDF) ---
    print("\n🗺️ Création de la carte exemple...")
    try:
        # Charger juste le fichier du milieu (mois 150 ≈ année 12)
        fichier_milieu = fichiers_25ans[150]
        ds_carte = xr.open_dataset(fichier_milieu)
        ds_carte_masked = ds_carte.where(mask['mask'] == 1)

        # Prendre le premier timestep
        ds_carte_final = ds_carte_masked.isel(valid_time=0)
        ds_carte_final.to_netcdf("carte_exemple_25ans.nc")

        print(f"✅ Fichier NetCDF créé : 'carte_exemple_25ans.nc'")
        print(f"   → Fichier source : {os.path.basename(fichier_milieu)}")

        ds_carte.close()

    except Exception as e_map:
        print(f"⚠️ Erreur lors de la création de la carte : {e_map}")

    # Fermeture du masque
    mask.close()

    print("\n🎉 Traitement terminé avec succès !")
    print(f"📁 Fichiers créés dans : {os.getcwd()}")

except Exception as e:
    print(f"\n❌ ERREUR BLOQUANTE : {e}")
    import traceback
    print(traceback.format_exc())